### Dataset

In [1]:
!pip install ucimlrepo

#### the libraries

In [169]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import adjusted_rand_score, silhouette_score
import warnings

### Feature Selection


In [127]:
t1 = target.values.ravel()
model = RandomForestRegressor(random_state=42)
model.fit(d_pca_df, target)

feature_importances = pd.Series(model.feature_importances_, index=d_pca_df.columns)
feature_importances = feature_importances.sort_values(ascending=False)
model_rfe = RandomForestClassifier(n_estimators=50, random_state=42)
X_final = in_scaled.copy()

n_features_to_select = 10
rfe = RFE(estimator=model_rfe, n_features_to_select=n_features_to_select)
rfe.fit(X_final, t1)
selected_features_rfe = X_final.columns[rfe.support_].tolist()
print("✅ Top 10 Features Selected by RFE:")
for i, feat in enumerate(selected_features_rfe, 1):
    print(f"   {i}. {feat}")

feature_ranking = pd.DataFrame({
    'Feature': X_final.columns,
    'Selected': rfe.support_,
    'Rank': rfe.ranking_
}).sort_values('Rank')

print("\n📊 Full Feature Ranking (Lowest Rank = Most Important):")
print(feature_ranking)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


✅ Top 10 Features Selected by RFE:
   1. age
   2. trestbps
   3. chol
   4. thalach
   5. exang
   6. oldpeak
   7. cp_4
   8. slope_2
   9. ca_1.0
   10. thal_7.0

📊 Full Feature Ranking (Lowest Rank = Most Important):
      Feature  Selected  Rank
0         age      True     1
2    trestbps      True     1
3        chol      True     1
5     thalach      True     1
7     oldpeak      True     1
6       exang      True     1
13    slope_2      True     1
10       cp_4      True     1
15     ca_1.0      True     1
19   thal_7.0      True     1
1         sex     False     2
16     ca_2.0     False     3
12  restecg_2     False     4
17     ca_3.0     False     5
4         fbs     False     6
9        cp_3     False     7
18   thal_6.0     False     8
14    slope_3     False     9
8        cp_2     False    10
11  restecg_1     False    11


In [128]:
scaler_chi2 = MinMaxScaler()
X_chi2 = scaler_chi2.fit_transform(X_final)


X_chi2 = np.clip(X_chi2, 0, None)

chi2_stats, p_values = chi2(X_chi2, t1)

chi2_results = pd.DataFrame({
    'Feature': X_final.columns,
    'Chi2_Stat': chi2_stats,
    'P_Value': p_values
}).sort_values('Chi2_Stat', ascending=False)

print("✅ Chi-Square Test Results (Most Significant First):")
print(chi2_results)

significant_features = chi2_results[chi2_results['P_Value'] < 0.05]
print(f"\n🎯 Number of statistically significant features (p < 0.05): {len(significant_features)}")
print("Top 5 significant features:")
print(significant_features.head(5)[['Feature', 'Chi2_Stat', 'P_Value']])

✅ Chi-Square Test Results (Most Significant First):
      Feature  Chi2_Stat       P_Value
19   thal_7.0  46.721875  1.742480e-09
10       cp_4  45.129615  3.736737e-09
6       exang  41.534482  2.083076e-08
16     ca_2.0  34.648382  5.485601e-07
17     ca_3.0  29.999871  4.894734e-06
13    slope_2  22.408432  1.661808e-04
9        cp_3  21.974965  2.027333e-04
8        cp_2  16.942450  1.983352e-03
15     ca_1.0  15.262296  4.187001e-03
18   thal_6.0  10.943440  2.720708e-02
1         sex   7.499223  1.117436e-01
4         fbs   6.658223  1.550901e-01
11  restecg_1   6.557266  1.612178e-01
12  restecg_2   6.458947  1.673955e-01
14    slope_3   5.528483  2.372371e-01
7     oldpeak   5.475394  2.419009e-01
5     thalach   3.133976  5.356604e-01
0         age   1.244886  8.706545e-01
2    trestbps   0.622014  9.605862e-01
3        chol   0.175931  9.963506e-01

🎯 Number of statistically significant features (p < 0.05): 10
Top 5 significant features:
     Feature  Chi2_Stat       P_Value
